<a href="https://colab.research.google.com/github/bahbi/emd-desafio-junior-data-scientist/blob/main/Visualizacao_analisePython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pywaffle

In [ ]:
# @title Setup code
!pip install -q streamlit
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
# import subprocess
# subprocess.Popen(["./cloudflared-linux-amd64", "tunnel", "--url", "http://localhost:8501"])
!nohup /content/cloudflared-linux-amd64 tunnel --url http://localhost:8501 &

In [ ]:
!grep -o 'https://.*\.trycloudflare.com' nohup.out | head -n 1 | xargs -I {} echo "Your tunnel url {}"

Your tunnel url https://priest-parallel-tough-sonic.trycloudflare.com


In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
#importante notificar que as vezes é necessário rodar mais de uma vez para funcionar
#lembrar dos arquivos

In [ ]:
"""%%writefile app.py
import streamlit as st

x = st.slider('Select a value')
st.write(x, 'squared is', x * x)

In [ ]:
#esse é o oficial

%%writefile app.py

import streamlit as st
import pandas as pd
import plotly.express as px

# Carregar os dados
df1 = pd.read_csv('/content/chamados_parte1.csv', delimiter=',')
df_bairros = pd.read_csv('/content/bairros_rj.csv', delimiter=',')

# Selecionar os dados relevantes
df1_selecionado = df1[['id_chamado', 'data_inicio', 'id_bairro', 'tipo',
                       'subtipo', 'status', 'longitude', 'latitude', 'reclamacoes', 'geometry']]
df_selecionado_bairros = df_bairros[['id_bairro', 'nome', 'subprefeitura']]
df_uniao_chamados_bairros = pd.merge(df1_selecionado, df_selecionado_bairros, on='id_bairro', how='left')

# Adicionar um menu lateral com opções para os títulos das seções
option = st.sidebar.selectbox('Selecione uma opção:',
                              ['Visualização chamados 1746 - em 01/04/2023',
                               'Visualização chamados de PS geral - em 01/04/2023'
                               ]) #'Visualização chamados relacionados ao período dos eventos - em 01/04/2023'

if option == 'Visualização chamados 1746 - em 01/04/2023':
    # Contar ocorrências por tipo
    st.subheader('Visualização chamados 1746 - em 01/04/2023')


    ocorrencias_por_tipo = df_uniao_chamados_bairros['tipo'].value_counts()
    df_ocorrencias_por_tipo = pd.DataFrame({'tipo': ocorrencias_por_tipo.index, 'contagem': ocorrencias_por_tipo.values})

    # Criar um gráfico de barras com o Plotly Express
    fig_tipo = px.bar(df_ocorrencias_por_tipo.head(10), x='contagem', y='tipo', orientation='h',
                      labels={'contagem': 'Quantidade', 'tipo': 'Tipo de Ocorrência'},
                      title='Top 10 Tipos de Ocorrências')

    # Personalizar as cores do gráfico
    fig_tipo.update_traces(marker_color='rgb(253, 180, 98)')

    # Exibir o gráfico com Streamlit
    st.plotly_chart(fig_tipo)

    # Contar ocorrências por status
    ocorrencias_por_status = df_uniao_chamados_bairros['status'].value_counts()
    df_ocorrencias_por_status = pd.DataFrame({'status': ocorrencias_por_status.index, 'contagem': ocorrencias_por_status.values})

    # Criar um gráfico de barras com o Plotly Express
    fig_status = px.bar(df_ocorrencias_por_status, x='contagem', y='status',
                        labels={'contagem': 'Quantidade', 'status': 'Status da Ocorrência'},
                        title='Ocorrências por Status')

    # Personalizar as cores do gráfico
    fig_status.update_traces(marker_color='rgb(253, 180, 98)')

    # Exibir o gráfico com Streamlit
    st.plotly_chart(fig_status)


    #mapa -1

    # Criar o gráfico de dispersão com Mapbox usando Plotly Express
    fig_mapa1 = px.scatter_mapbox(df_uniao_chamados_bairros,
                            lat='latitude',
                            lon='longitude',
                            color='tipo',
                            hover_name='nome',
                            zoom=10,
                            mapbox_style="carto-positron",
                            center=dict(lat=-22.9068467, lon=-43.1728965),
                            title='Pontos de Chamados por Bairro')

    # Exibir o gráfico com Streamlit
    st.plotly_chart(fig_mapa1)


    # Contar ocorrências por subprefeitura e tipo
    df_count = df_uniao_chamados_bairros.groupby(['subprefeitura', 'tipo']).size().reset_index(name='count')

    # Criar o gráfico de barras empilhadas com Plotly Express
    fig_subp_tipo = px.bar(df_count,
                x='subprefeitura',
                y='count',
                color='tipo',
                title='Tipos de Chamados por Subprefeitura',
                labels={'count': 'Número de Chamados'},
                barmode='stack')

    # Exibir o gráfico com Streamlit
    st.plotly_chart(fig_subp_tipo)


    #-----------------------------------------------

else:
      #option == 'Visualização chamados de PS geral - em 01/04/2023'
    # Mapa -1
    # Adicionar título para a seção 2
    st.subheader('Visualização chamados de PS geral  - em 01/04/2023 ')

    df_filtrado1 = pd.read_csv('/content/chamados_parte2.csv', delimiter=',')

    df2_selecionado = df_filtrado1[['id_chamado', 'data_inicio','id_bairro','categoria','tipo',
                          'subtipo','status','longitude','latitude','reclamacoes','geometry']]

    df3 = pd.read_csv('/content/hoteleira_ocupacao.csv', delimiter=',')

    df_uniao_chamados_bairros_PS = pd.merge(df2_selecionado, df_selecionado_bairros, on='id_bairro', how='left')


    # Contar ocorrências por latitude e longitude
    df_count2 = df_uniao_chamados_bairros_PS.groupby(['latitude', 'longitude']).size().reset_index(name='count')

    # Criar o gráfico de densidade com Mapbox usando Plotly Express
    fig_heatmap = px.density_mapbox(df_count2,
                                    lat='latitude',
                                    lon='longitude',
                                    z='count',
                                    radius=10,
                                    center=dict(lat=-22.9068467, lon=-43.1728965),
                                    zoom=11,
                                    mapbox_style="carto-positron",
                                    title='Mapa de Calor de Chamados por Bairro')

    # Exibir o gráfico com Streamlit
    st.plotly_chart(fig_heatmap)


    # Contar ocorrências por status
    ocorrencias_por_status1 = df_uniao_chamados_bairros_PS['status'].value_counts()
    df_ocorrencias_por_status1 = pd.DataFrame({'status': ocorrencias_por_status1.index, 'contagem': ocorrencias_por_status1.values})

    # Selecionar as top 9 ocorrências
    barras_qtd = 9
    df_top = df_ocorrencias_por_status1.head(barras_qtd)

    # Criar o gráfico de barras com Plotly Express
    fig = px.bar(df_top, x='contagem', y='status', orientation='h',
                labels={'contagem': 'Quantidade', 'status': 'Status da Ocorrência'},
                title='Top 9 Ocorrências por Status')

    # Exibir o gráfico com Streamlit
    st.plotly_chart(fig)


    # Converter a coluna de data para o formato datetime
    df_uniao_chamados_bairros_PS['data_inicio'] = pd.to_datetime(df_uniao_chamados_bairros_PS['data_inicio'])

    # Agrupar o número de chamados por data
    df_tendencia = df_uniao_chamados_bairros_PS.groupby(df_uniao_chamados_bairros_PS['data_inicio'].dt.date).size().reset_index(name='id_chamado')

    # Criar o gráfico de linha com Plotly Express
    fig = px.line(df_tendencia,
                  x='data_inicio',
                  y='id_chamado',
                  title='Tendência de Chamados ao Longo do Tempo')

    # Personalizar os rótulos dos eixos
    fig.update_xaxes(title_text='Data')
    fig.update_yaxes(title_text='Número de Chamados')

    # Exibir o gráfico com Streamlit
    st.plotly_chart(fig)

    #-----------------------------------------------

#else:
    # Adicionar título para a seção 3
    st.subheader('Visualização chamados relacionados ao período dos eventos - em 01/04/2023')

    # Converter a coluna de data para o formato datetime
    df_uniao_chamados_bairros_PS['data_inicio'] = pd.to_datetime(df_uniao_chamados_bairros_PS['data_inicio'])

    # Restante do código para a terceira seção
    # ...
    df_chamados = pd.DataFrame(df_uniao_chamados_bairros_PS)
    df_categorias = pd.DataFrame(df3)


    df_chamados['data_inicio'] = pd.to_datetime(df_chamados['data_inicio'])
    df_categorias['data_inicial'] = pd.to_datetime(df_categorias['data_inicial'])
    df_categorias['data_final'] = pd.to_datetime(df_categorias['data_final'])


    def verifica_correspondencia(data_inicio, data_inicial, data_final):
        return (data_inicio >= data_inicial) & (data_inicio <= data_final)


    chamados_correspondentes = []
    for _, chamado in df_chamados.iterrows():
        for _, categoria in df_categorias.iterrows():
            if verifica_correspondencia(chamado['data_inicio'], categoria['data_inicial'], categoria['data_final']):
                chamados_correspondentes.append((chamado['id_chamado'], categoria['evento'], chamado['data_inicio']))
                break


    df_correspondencias = pd.DataFrame(chamados_correspondentes, columns=['id_chamado', 'evento', 'data_inicio'])

    # Calcular o número de aparições de cada evento
    df_correspondencias['numero_aparicoes'] = df_correspondencias.groupby(['id_chamado', 'evento'])['data_inicio'].transform('count')

    # Contar o número de aparições de cada evento
    contagem_categorias = df_correspondencias['evento'].value_counts().reset_index()
    contagem_categorias.columns = ['evento', 'numero_aparicoes']

    # Criar o gráfico de pizza com Plotly Express
    fig_ap_ev1 = px.pie(contagem_categorias, values='numero_aparicoes', names='evento',
                title='Número de Aparições por Evento')

    # Personalizar as cores do gráfico
    fig_ap_ev1.update_traces(marker_colors=px.colors.qualitative.Pastel)

    # Exibir o gráfico com Streamlit
    st.plotly_chart(fig_ap_ev1)

    df_final = pd.merge(df_correspondencias, df_uniao_chamados_bairros_PS, on='id_chamado', how='left')


    # Agrupar os dados por evento e status e contar o número de ocorrências
    df_count1 = df_final.groupby(['evento', 'status']).size().reset_index(name='count')

    # Criar o gráfico de barras empilhadas com Plotly Express
    fig_n = px.bar(df_count1,
                x='evento',
                y='count',
                color='status',
                title='Tipos de Chamados por Subprefeitura',
                labels={'count': 'Número de Chamados'},
                barmode='stack')

    # Exibir o gráfico com Streamlit
    st.plotly_chart(fig_n)

    #import streamlit as st
    #import plotly.express as px

    # Criar o gráfico de dispersão com Mapbox usando Plotly Express
    fig_m1 = px.scatter_mapbox(df_final,
                            lat='latitude',
                            lon='longitude',
                            color='evento',
                            hover_name='nome',
                            zoom=10,
                            mapbox_style="carto-positron",
                            center=dict(lat=-22.9068467, lon=-43.1728965),
                            title='Pontos de Chamados por evento')

    # Exibir o gráfico com Streamlit
    st.plotly_chart(fig_m1)





Overwriting app.py
